In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import numpy as np
import torch
import torchio as tio
import h5py
from ipywidgets import interact
import matplotlib.pyplot as plt

dir2 = os.path.abspath('..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

In [2]:
# classify stimulus images with a imagenet model

import torchvision.models as models
import torchvision.transforms as T
from pathlib import Path
import h5py

root = 'X:\\Datasets\\Deep-Image-Reconstruction'
stimulus_images = h5py.File(Path(root) / "derivatives" / "stimulus_images.hdf5", "r")

model = models.resnet152(pretrained=True)
model.eval()
model.cuda()
print()

In [3]:
import torch.nn.functional as F

transform = T.Compose([
    T.Resize(224),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

y_pred = []
for stimulus_id in stimulus_images.keys():
    x = stimulus_images[stimulus_id]['data'][:]
    x = torch.from_numpy(x).float() / 255
    x = x.permute(2, 0, 1)
    x = transform(x)
    x = x.cuda()[None]
    with torch.no_grad():
        y = model(x)
    y_pred.append(y)
y_pred = torch.cat(y_pred)

In [4]:
# get wordnet averages

averages = {}
for i, stimulus_id in enumerate(stimulus_images.keys()):
    y = y_pred[i]
    if '.' in stimulus_id:
        wordnet_id = stimulus_id.split('.')[0]
        if wordnet_id not in averages:
            averages[wordnet_id] = [y]
        else:
            averages[wordnet_id].append(y)
    else:
        averages[stimulus_id] = [y]

averages = {k: torch.stack(v).mean(axis=0) for k, v in averages.items()}

y_pred_averaged = []
for stimulus_id in stimulus_images.keys():
    if '.' in stimulus_id:
        wordnet_id = stimulus_id.split('.')[0]
        y_pred_averaged.append(averages[wordnet_id])
    else:
        y_pred_averaged.append(averages[stimulus_id])
y_pred_averaged = torch.stack(y_pred_averaged)

In [15]:
# look at results

import torch.nn.functional as F
from research.imagenet_classes import imagenet_classes

@interact(i=(0, 1299), average=False)
def compare(i, average):
    stimulus_id = list(stimulus_images.keys())[i]
    x = stimulus_images[stimulus_id]['data'][:]
    y = y_pred_averaged if average else y_pred
    y = F.softmax(y, dim=1)
    y = y[i]
    plt.imshow(x)
    
    top_5 = [i.item() for i in list(y.argsort()[-5:].cpu())]
    top_5.reverse()
    results = {imagenet_classes[i]: y[i].cpu().item() for i in top_5}
    results = {k.split(',')[0]: round(v, 3) for k, v in results.items()}
    print(results)

interactive(children=(IntSlider(value=649, description='i', max=1299), Checkbox(value=False, description='aver…

In [20]:
# Make x_train for inversion dataset

import h5py
from pathlib import Path
from scipy import ndimage
from PIL import Image

root = 'X:\\Datasets\\Deep-Image-Reconstruction'
stimulus_images = h5py.File(Path(root) / "derivatives" / "stimulus_images.hdf5", "r")

transform = T.Compose([
    T.Resize(224),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

xtrain = []
for stimulus_id in stimulus_images.keys():
    x = stimulus_images[stimulus_id]['data'][:]
    x = T.ToPILImage()(x)
    x = T.Resize(256)(x)
    xtrain.append(np.array(x))
xtrain = np.stack(xtrain)
xtrain = torch.from_numpy(xtrain)
xtrain = xtrain.permute(0, 3, 1, 2)

In [12]:
y_pred_averaged.mean(dim=0).shape

torch.Size([1000])

In [21]:
from pathlib import Path
import h5py

out_dir = Path('G:\\Github Repositories\\exploiting-gan-internal-capacity\\inverses')

with h5py.File(out_dir / 'kamitani_stimulus256.hdf5', 'w') as f:
    ytrain = F.softmax(y_pred_averaged, dim=1)
    f.create_dataset('xtrain', data=xtrain)
    f.create_dataset('ytrain', data=ytrain.cpu())

In [ ]:
y_pred_averaged.shape